<a href="https://colab.research.google.com/github/arockingvirus/test/blob/master/pinto_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font color = red> PTVA Algorithm

import community as community_louvain
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import numpy as np
import random
import ndlib.models.ModelConfig as mc
import ndlib.models.epidemics as ep
import matplotlib.pyplot as plt
import pprint

G=nx.Graph()
# gr=pd.read_csv('karate.csv')
#gr=pd.read_csv('dolphin.csv')
gr=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/football.csv")
edge=[(gr['Source'][i],gr['Destination'][i]) for i in range(0,len(gr['Source']))]
G.add_edges_from(edge)
node=max([max(x[0],x[1]) for x in edge])
# compute the best partition
# partition contains color of each node
partition = community_louvain.best_partition(G)

# visualization of clusters
pos = nx.spring_layout(G)
cmap = cm.get_cmap('viridis', max(partition.values()) + 1)
nx.draw_networkx_nodes(G, pos, partition.keys(), node_size=200,cmap=cmap, node_color=list(partition.values()),)
nx.draw_networkx_edges(G, pos, alpha=0.5)
plt.show()

In [ ]:
import community as community_louvain
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import numpy as np
import random
import ndlib.models.ModelConfig as mc
import ndlib.models.epidemics as ep
import matplotlib.pyplot as plt
import pprint

ModuleNotFoundError: ignored

In [ ]:
import collections
import itertools
#import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import random
import operator
import math
import networkx as nx
import numpy as np
## import PTVA_LI_2.source_est_tools as tl
import operator
import collections

import scipy.stats as st
from scipy.special import logsumexp

In [ ]:
from scipy.special import logsumexp


In [ ]:
# Algorithms used for rumor source inference in a graph
# - Pinto algorithm

import networkx as nx
import numpy
# from numpy import matrix, array
# from numpy import linalg
import math
import pandas as pd
import matplotlib.pyplot as plt


def observed_delay(g, O):
    d = numpy.zeros(shape=(len(O) - 1, 1))
    for i in range(len(O) - 1):
        d[i][0] = g.nodes[O[i + 1]]['time'] - g.nodes[O[i]]['time']
    return d


def delay_covariance(T, O, sigma2):
    """
    Computes lambda
    :param T: tree
    :param O: list of observers
    :param sigma2: variance
    :return:
    """
    # TODO stop using all_simple_paths (complexity)
    n = len(O)
    delta = numpy.zeros(shape=(n - 1, n - 1))
    T = T.to_undirected()
    for k in range(n - 1):
        for i in range(n - 1):
            if i == k:
                delta[k][i] = len(list(nx.all_simple_paths(T, O[0], O[k + 1]))[0]) - 1
            else:
                c1 = list(nx.all_simple_paths(T, O[0], O[k + 1]))[0]
                c2 = list(nx.all_simple_paths(T, O[0], O[i + 1]))[0]
                S = [x for x in c1 if x in c2]
                delta[k][i] = len(S) - 1
    delta = delta * (sigma2 ** 2)  # FIXME : square or not ?
    return delta


def deterministic_delay(T, s, O, mi):
    """
    Computes mu_s
    :param T: tree
    :param s: source
    :param O: list of observers
    :param mi: mean
    :return:
    """
    constant = height_node(T, s, O[0])
    mi_s = numpy.zeros(shape=(len(O) - 1, 1))
    for i in range(len(O) - 1):
        mi_s[i][0] = height_node(T, s, O[i + 1]) - constant
    mi_s = mi * mi_s
    return mi_s


def height_node(T, s, node):
    l = list(nx.all_simple_paths(T, s, node))
    if l == []:
        return 0
    else:
        return len(l[0]) - 1


def run(g, O, mi, sigma2, k0):
    """
    Main
    :param g: graph
    :param O: list of observers <<<< ACTIVE observers !
    :param mi: mean
    :param sigma2: variance
    :return:
    """
    # TODO : consider only active observers !
    first_node = O[0]

    # Compute the delay vector d relative to first_node
    d = observed_delay(g, O)

    # Score list of pair (node , score)
    s = {}
    # s=[(O[0],0)]

    # initialise (node , score)
    v = [O[0], 0]
    flag = 0

    # max score
    s_max = 0

    while v[1] >= s_max:
        Tv = {}
        Tv[v[0]] = 0
        for n in g[v[0]]:
            if n not in s.keys():
                tree_bfs = nx.bfs_tree(g, source=first_node)
                mu_s = deterministic_delay(tree_bfs, n, O, mi)
                delta = delay_covariance(tree_bfs, O, sigma2)
                inverse = numpy.linalg.inv(delta)
                score = (numpy.exp(-.5 * numpy.dot(numpy.dot((d - mu_s).T, inverse), (d - mu_s)))) / (
                    numpy.sqrt(abs(numpy.linalg.det(delta))))
                Tv[n] = score[0][0]
        if len(Tv) != 0:
            neighbor_n = [a for a in g[v[0]]]

            if v[0] not in Tv:
                curr_score_node = s[v[0]]
            else:
                curr_score_node = Tv[v[0]]
            curr_node = v[0]

            change = 0

            for a in neighbor_n:
                if a in Tv:
                    neigh_score_a = Tv[a]
                else:
                    continue

                if curr_score_node < neigh_score_a:
                    change = 1
                    curr_node = a

            if change != 0:
                s.update(Tv)
                v = [curr_node, s[curr_node]]
                s_max = v[1]

            else:
                break

        # end of if
    # end of while

    sorted_score = sorted(s.items(), key=lambda kv: kv[1], reverse=True)

    print("The rumour source in order : ")
    for a in range(3):
        print(sorted_score[a][0], "\t")

    return s


def from_pandas_dataframe(df, source, target, edge_attr=None, create_using=None):
    g = nx.Graph()

    # Index of source and target
    src_i = df.columns.get_loc(source)
    tar_i = df.columns.get_loc(target)
    if edge_attr:
        # If all additional columns requested, build up a list of tuples
        # [(name, index),...]
        if edge_attr is True:
            # Create a list of all columns indices, ignore nodes
            edge_i = []
            for i, col in enumerate(df.columns):
                if col is not source and col is not target:
                    edge_i.append((col, i))
        # If a list or tuple of name is requested
        elif isinstance(edge_attr, (list, tuple)):
            edge_i = [(i, df.columns.get_loc(i)) for i in edge_attr]
        # If a string or int is passed
        else:
            edge_i = [(edge_attr, df.columns.get_loc(edge_attr)), ]

        # Iteration on values returns the rows as Numpy arrays
        for row in df.values:
            g.add_edge(row[src_i], row[tar_i], {i: row[j] for i, j in edge_i})

    # If no column names are given, then just return the edges.
    else:
        for row in df.values:
            g.add_edge(row[src_i], row[tar_i])
    return g




####################################################################################
numpy.random.seed(54)  # option for reproducibility---- 51,52,53,54,55,57,58,59
filename = "football"
df = pd.read_csv("/Users/sahajsingh/Downloads/football.csv")
df = df[["Source", "Destination"]]
g = from_pandas_dataframe(df, source='Source', target='Destination')
nx.draw(g,with_labels = True)
node_len = len(g.nodes)
plt.savefig(filename+"_PTVA1.png")
plt.show()
# ############################################################ Change here
k0 = math.ceil(math.sqrt(len(g)))




In [ ]:
#G = nx.karate_club_graph()
G=g
timeOfDiffusions=[]
# Model selection - diffusion time
model = ep.SIModel(G)

# Model Configuration
cfg = mc.Configuration()
cfg.add_model_parameter('beta', 0.03)
cfg.add_model_parameter("fraction_infected", 0.01)
model.set_initial_status(cfg)

# Simulation execution
iterations = model.iteration_bunch(200)

#Mapping diffusion_time_to_each_node
time_of_diffusion={}
for i in range(1,len(G)):
    time_of_diffusion[i]=-1
for i in iterations:
    for j in i['status']:
        if(i['status'][j]==1):
            time_of_diffusion[j]=i['iteration']
timeOfDiffusions.append(len(time_of_diffusion))

In [ ]:
time_of_diffusion

In [ ]:
sorted_values = sorted(time_of_diffusion.values()) # Sort the values
sorted_dict = {}

for i in sorted_values:
    for k in time_of_diffusion.keys():
        if time_of_diffusion[k] == i:
            sorted_dict[k] = time_of_diffusion[k]

print(sorted_dict)

In [ ]:
j=0
O = numpy.random.randint(low=1, high=len(G), size=k0).tolist()

In [ ]:
j=0
t=[]
for i in O:
    if(time_of_diffusion[i]!=-1):
        t.append(time_of_diffusion[i])

In [ ]:
O

In [ ]:
t

In [ ]:
# ###################################################################################
# O = numpy.random.randint(low=1, high=node_len, size=k0).tolist()
# t = numpy.random.uniform(low=0.000000001, high=6, size=k0).tolist()
print(len(g.nodes))
print(g.nodes)
for a in range(0, k0):
    g.nodes[O[a]]['time'] = t[a]
mi = numpy.mean(t)
sigma2 = numpy.var(t)

# #####Running the algorithm:
# run(g, O, mi, sigma2,k0)
score = run(g, O, mi, sigma2, k0)
score = sorted(score.items(), key=lambda kv: kv[1], reverse=True)
score = [list(a)[0] for a in g.nodes(data=True)]

# ################################# Coloring nodes
node_color = []
count = 0
score_list = score[0:5]
for i in range(1, node_len + 1):
    # if the node has the attribute group1
    # elif list(node)[0] in score[:5]:
    if i in O:
        node_color.append('green')
    elif i in score_list:
        node_color.append('red')
    elif 5 == 5:
        count = count + 1
        node_color.append('black')
nx.draw(g,with_labels = True, node_size=200, node_color=node_color,
        alpha=1, linewidths=0.5, width=0.5, edge_color='black')
plt.savefig(filename+"_PTVA.png")
plt.show()

In [ ]:
def accuracy(source_node,predicted_set,score_set):
    a = []
    for i in predicted_set:
        for j in true_set:
            if i==j:
                a.append(1/score_set[i])
                print(score_set[i])
    acc = numpy.mean(a)
    return acc

In [ ]:
score1 = run(g, O, mi, sigma2, k0)
print(score1)
score1 = sorted(score1.items(), key=lambda kv: kv[1], reverse=True)

In [ ]:
accuracy([47,22,48,50,54],[54,50,68],score1)